__Files__

__assessments.csv__ — This file contains information about the grades in the test. Typically, each course in a semester includes a series of tests with grades, followed by a final exam test (exam).

code_module — course ID code.

code_presentation — semester (Identification code).

id_assessment — test (Assessment ID number).

assessment_type — type of test. There are three types of assessment: teacher assessment (TMA), computer-based assessment (CMA), and course exam (Exam).

date — information about the final date of the test. Calculated as the number of days since the beginning of the semester. The start date of the semester has a number 0 (zero).

weight — weight of the test in % of the course grade. Usually exams are considered separately and have a weight of 100%; the sum of all other grades is 100%.

__courses.csv__ — the file contains a list of courses by semesters.

code_module — course (identification code).

code_presentation — semester (identification code).

module_presentation_length — semester length in days.

__studentAssessment.csv__ — This file contains the results of student tests. If the student does not submit the work for evaluation, the result is not written to the table.

id_assessment — test (identification number).

id_student — the identification number of the student.

date_submitted — the date the student submitted the test, measured as the number of days since the beginning of the semester.

is_banked — the fact that the test was retaken from the previous semester (sometimes courses are retaken by students who have returned from sabbatical leave).

score — the student's score in this test. The range is from 0 to 100. A score below 40 fails/unsuccessfully passes the test.

__studentRegistration.csv__ — this file contains information about the time the student registered for the course in the semester.

code_module — course (identification code).

code_presentation — semester (identification code)

id_student — identification number of the student.

date_registration — the date of the student's registration. This is the number of days measured from the beginning of the semester (e.g., a negative value of -30 means that the student registered for the course 30 days before it started).

date_unregistration — the date the student deregistered from the course. For students who have completed the course, this field remains blank.

In [132]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

In [2]:
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessment = pd.read_csv('studentAssessment.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

__1. How many students have successfully completed only one course?__  (A successful pass is a course credit on an exam.)

In [5]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [8]:
studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [3]:
# I choose only assessements with type "Exam".
df_exam = assessments.query('assessment_type == "Exam"')
df_exam.head()
df_exam.shape

(24, 6)

In [7]:
df_exam.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0


In [4]:
df_exam.id_assessment.nunique()

24

In [5]:
# I only take the assessements that were successfully passed.
studentAssessment_40 = studentAssessment.query('score >= 40')
studentAssessment_40.shape

(166161, 5)

In [6]:
full = studentAssessment_40.merge(df_exam, how='inner', on='id_assessment')

In [10]:
full.shape

(4392, 10)

In [7]:
# I count the number of unique successful exams for each student.
full = full.groupby('id_student', as_index=False).agg({'score':'count'})

In [37]:
full.head()

,id_student,score
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1


In [8]:
one = full.query('score == 1')

In [43]:
one.id_student.nunique()

3802

3802 students have successfully completed only one course.

__2. Identify the hardest and easiest exams: Find the courses and exams within the course that have the lowest and highest completion rates.__ Completion rate = number of successful exams / number of all attempts to take the exam.

In [9]:
df_exam = assessments.query('assessment_type == "Exam"')
success = studentAssessment.merge(df_exam, how='inner', on='id_assessment')

In [13]:
success.shape

(4959, 10)

In [14]:
success.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0
1,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
2,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
3,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
4,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0


In [10]:
final = success.groupby('id_assessment', as_index = False).agg({'id_student': 'count'}).rename(columns={'id_student': 'number'})

In [16]:
final

,id_assessment,number
0,24290,747
1,24299,1168
2,25340,602
3,25354,968
4,25361,524
5,25368,950


In [11]:
passed = success.query('score >=40').groupby('id_assessment', as_index=False)\
        .agg({'id_student': 'count'}).rename(columns={'id_student': 'done'})

In [18]:
passed

,id_assessment,done
0,24290,664
1,24299,1019
2,25340,504
3,25354,878
4,25361,485
5,25368,842


In [12]:
all = final.merge(passed, how='inner', on='id_assessment')
all

,id_assessment,number,done
0,24290,747,664
1,24299,1168,1019
2,25340,602,504
3,25354,968,878
4,25361,524,485
5,25368,950,842


In [13]:
all['completion'] = (all.done/all.number).round(2)

In [14]:
all = all.sort_values(by='completion').reset_index(drop=True)

In [22]:
all

,id_assessment,number,done,completion
0,25340,602,504,0.84
1,24299,1168,1019,0.87
2,24290,747,664,0.89
3,25368,950,842,0.89
4,25354,968,878,0.91
5,25361,524,485,0.93


In [15]:
print('the most challenging exam has id', all.id_assessment[0])
print('the easiest exam has id:', list(all.id_assessment)[-1])

the most challenging exam has id 25340
the easiest exam has id: 25361


__3. For each course, determine the average time for passing the exams (by passing we mean the last successful completion of the exam by the student).__

In [16]:
df_exam = assessments.query('assessment_type == "Exam"')
# I exclude the students who have returned from sabbatical leave
full_3 = studentAssessment.query('is_banked ==0').merge(df_exam, how='inner', on='id_assessment')
full_3

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0
1,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
2,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
3,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
4,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...
4954,25368,652680,243,0,91.0,DDD,2014J,Exam,NaN,100.0
4955,25368,652732,249,0,47.0,DDD,2014J,Exam,NaN,100.0
4956,25368,652965,242,0,87.0,DDD,2014J,Exam,NaN,100.0
4957,25368,653051,242,0,27.0,DDD,2014J,Exam,NaN,100.0


In [17]:
full_3 = full_3.groupby('code_module', as_index=False).agg({'date_submitted':'mean'})

In [93]:
full_3

,code_module,date_submitted
0,CCC,239.408877
1,DDD,237.901445


__4. Identify the most popular courses (Top 3) by the number of registrations for them. And also the courses with the highest churn rate (Top 3).__

In [18]:
# 1. For every semester in the table sudentRegistration I count the number of unique registered students.
# 2. For every semester in the table sudentRegistration I count the number of unique students who canceled the registration.
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [19]:
regs = studentRegistration\
    .groupby('code_module', as_index=False).id_student.nunique()\
    .sort_values('id_student', ascending=False).rename(columns={'id_student': 'registration_number'}).head(3)

In [29]:
print(regs)

  code_module  registration_number
1         BBB                 7692
5         FFF                 7397
3         DDD                 5848


In [20]:
studentRegistration['unreg'] = studentRegistration.date_unregistration.isna()

In [37]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration,unreg
0,AAA,2013J,11391,-159.0,NaN,True
1,AAA,2013J,28400,-53.0,NaN,True
2,AAA,2013J,30268,-92.0,12.0,False
3,AAA,2013J,31604,-52.0,NaN,True
4,AAA,2013J,32885,-176.0,NaN,True


In [21]:
churn = studentRegistration.query('unreg == False')\
    .groupby('code_module', as_index=False).id_student.nunique()\
    .sort_values('id_student', ascending=False).rename(columns={'id_student': 'unregistration_number'}).head(3)

In [39]:
print(churn)

  code_module  unregistration_number
1         BBB                   2314
5         FFF                   2249
3         DDD                   2065


__5. Write a function that allows you to construct a cohort (semester) analysis.
Between early 2013 and late 2014, identify the semester with the lowest course completion rate and the longest average course exams time.__

What is needed for a cohort analysis?
Before conducting a cohort analysis, four parameters are determined:

1. The sign of cohort formation - the action that brings people together in a group: first visit, purchase, installation, registration, etc.
2. Cohort size - time interval: day, week, month.
3. reporting period - time of group behavior research.
4. Analyzed key indicator: ROI, Retention Rate, LTV, etc.

In [22]:
assessments_2 = assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']].drop_duplicates()

In [72]:
assessments_2.shape

(24, 2)

In [23]:
studentAssessment_2 = studentAssessment.query('is_banked == 0')

In [24]:
new = studentAssessment_2.merge(assessments_2, how='inner', on='id_assessment')

In [75]:
new

,id_assessment,id_student,date_submitted,is_banked,score,code_presentation
0,24290,558914,230,0,32.0,2014B
1,24290,559706,234,0,78.0,2014B
2,24290,559770,230,0,54.0,2014B
3,24290,560114,230,0,64.0,2014B
4,24290,560311,234,0,100.0,2014B
...,...,...,...,...,...,...
4954,25368,652680,243,0,91.0,2014J
4955,25368,652732,249,0,47.0,2014J
4956,25368,652965,242,0,87.0,2014J
4957,25368,653051,242,0,27.0,2014J


In [25]:
df_5 = new.assign(success=lambda x: (x.score >= 40).astype(int),
                success_date_submitted = lambda x: x.success * x.date_submitted)

In [77]:
df_5.dtypes

id_assessment               int64
id_student                  int64
date_submitted              int64
is_banked                   int64
score                     float64
code_presentation          object
success                     int64
success_date_submitted      int64
dtype: object

In [40]:
df_5.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_presentation,success,success_date_submitted
0,24290,558914,230,0,32.0,2014B,0,0
1,24290,559706,234,0,78.0,2014B,1,234
2,24290,559770,230,0,54.0,2014B,1,230
3,24290,560114,230,0,64.0,2014B,1,230
4,24290,560311,234,0,100.0,2014B,1,234


In [46]:
df_5.query('id_student=="560311"')

,id_assessment,id_student,date_submitted,is_banked,score,code_presentation,success,success_date_submitted
4,24290,560311,234,0,100.0,2014B,1,234
2415,25340,560311,230,0,73.0,2013B,1,230


In [26]:
df_5_final = df_5.groupby('code_presentation', as_index=False)\
    .agg({'success': 'mean', 'success_date_submitted': [lambda x: np.average(x, weights=(x>0))]})

In [83]:
df_5_final

,code_presentation,success,success_date_submitted
,,mean,<lambda>
0,2013B,0.837209,230.164683
1,2013J,0.907025,239.509112
2,2014B,0.904013,232.997389
3,2014J,0.878659,243.680279


The semester with the lowest course completion rate is 2013B.

The semester with longest average course exams time  is 2014J.

__6. Segmentation-based approaches are often used for qualitative audience analysis. Using python, build adapted RFM clusters of students to qualitatively evaluate your audience.__

In the adapted clustering you can choose the following metrics:

R - average time per exam, 

F - course completion rate,

M - average number of points received per exam. 

Describe in detail how you created the clusters. For each RFM-segment build boundaries of recency, frequency and monetary metrics to interpret those clusters.

In [27]:
rfm_df = df_5.groupby('id_student', as_index=False)\
    .agg({'success_date_submitted': 'mean', 'success': 'mean', 'score': 'mean'}) \
    .rename(columns={'success_date_submitted': 'recency', 'success': 'frequency', 'score': 'monetary'})

In [28]:
rfm_df

,id_student,recency,frequency,monetary
0,23698,243.0,1.0,80.0
1,24213,236.0,1.0,58.0
2,27116,243.0,1.0,96.0
3,28046,237.0,1.0,40.0
4,28787,243.0,1.0,44.0
...,...,...,...,...
4628,2694886,236.0,1.0,69.0
4629,2694933,230.0,1.0,73.0
4630,2695608,237.0,1.0,73.0
4631,2697181,230.0,1.0,80.0


In [30]:
rfm_df['recency']=rfm_df['recency'].astype(int)

In [69]:
rfm_df.frequency.dtype

dtype('float64')

In [34]:
 df_5['id_student'].nunique()

4633

In [38]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])

In [39]:
quantiles

,id_student,recency,frequency,monetary
0.25,501158.0,230.0,1.0,50.0
0.50,588482.0,237.0,1.0,66.0
0.75,646351.0,243.0,1.0,82.0


In [95]:
# Recency and Monetary clusters I will define accourding the quantiles

def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4

def MClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [96]:
rfmSegmentation = rfm_df

In [199]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary'].apply(MClass, args=('monetary',quantiles))

def rfm_transition_freq(data, column_name):
    new_column = data[column_name].apply(lambda x: 1 if x == 1 else 2 if x == 0.5 else 3)
    data[f'rfm_{column_name}'] = new_column    
    
rfm_transition_freq(rfmSegmentation, "frequency")

In [200]:
rfmSegmentation

,id_student,recency,frequency,monetary,R_Quartile,M_Quartile,RFMClass,rfm_frequency
0,23698,243,1.0,80.0,3,2,312,1
1,24213,236,1.0,58.0,2,3,213,1
2,27116,243,1.0,96.0,3,1,311,1
3,28046,237,1.0,40.0,2,4,214,1
4,28787,243,1.0,44.0,3,4,314,1
...,...,...,...,...,...,...,...,...
4628,2694886,236,1.0,69.0,2,2,212,1
4629,2694933,230,1.0,73.0,1,2,112,1
4630,2695608,237,1.0,73.0,2,2,212,1
4631,2697181,230,1.0,80.0,1,2,112,1


In [192]:
# Combine all 3 clusters in one column 

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) + rfmSegmentation.rfm_frequency.map(str) + rfmSegmentation.M_Quartile.map(str)

In [193]:
pd.crosstab(index = rfmSegmentation.rfm_frequency, columns = rfmSegmentation.M_Quartile)

M_Quartile,1,2,3,4
rfm_frequency,,,,
1,1040,1242,1163,624
2,0,0,8,20
3,0,0,0,536


In [196]:
rfmSegmentation.RFMClass.value_counts().sort_values()

123      8
124     20
414    121
114    136
214    168
111    174
112    186
412    189
413    198
314    199
113    211
211    250
411    291
213    319
311    325
212    405
313    435
312    462
134    536
Name: RFMClass, dtype: int64

The most promising group is 111, it consists of 174 students. The students passed the exams with relative high number of points per exam (from 83 to 100) and within a shorter period of time (from 0 to 230 days).

__Cluster borders:__ 

R - average time per exam 
1. 0-230 days, 2. 231-237 days, 3. 238-243 days, 4. 244+ days.

F - course completion rate
1. exams passed, 2. some exams passed, 3. exams not passed.

M - average number of points received per exam
1. 83-100, 2. 67-82, 3. 51-66, 4. 0-50.